In [92]:
import torch
import numpy as np
import pandas as pd
import requests
import json
from torch import nn
from torch.nn.utils import rnn
from torch.utils.data import RandomSampler, BatchSampler

## Definining Model Architecture
Training on LSTM to try and capture some hidden state of a team

In [228]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.linear1 = nn.Linear(hidden_size, 64)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(64, 2)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        lstm_out, _ = self.lstm(x)
#         print(lstm_out.shape)
        dropout = self.dropout(lstm_out)
        linear1 = self.linear1(dropout)
        relud = self.relu(linear1)
        pred = self.linear2(relud)
        
        return pred

In [232]:
raw_df = pd.read_csv('2019pbpfeatures_calvin.csv')
raw_df.head(2)

,Date,URL,Location,Time,WinningTeam,Quarter,SecLeft,AwayPlay,HomePlay,AwayTeam,...,AwayMade 3-pt shotTotal,AwayTurnoverTotal,HomeTurnoverTotal,AwayFreeThrowTotal,HomeFreeThrowTotal,AwayMadeFreeThrowTotal,HomeMadeFreeThrowTotal,HomeFinalScore,AwayFinalScore,HomeWin
0,2019-10-22 00:00:00,/boxscores/201910220LAC.html,STAPLES Center Los Angeles California,2021-05-05 22:30:00,LAC,1,720,Jump ball: J. McGee vs. I. Zubac (L. James gai...,NaN,LAL,...,-3.236354,-3.551278,-3.529433,-3.181216,-3.117167,-2.94465,-2.847606,0.8,0.133333,1
1,2019-10-22 00:00:00,/boxscores/201910220LAC.html,STAPLES Center Los Angeles California,2021-05-05 22:30:00,LAC,1,707,L. James makes 2-pt layup from 3 ft (assist by...,NaN,LAL,...,-3.236354,-3.551278,-3.529433,-3.181216,-3.117167,-2.94465,-2.847606,0.8,0.133333,1


### Testing for Single Team
Picking the Warriors ('GSW') for now

In [188]:
# df = raw_df[(raw_df['HomeTeam'] == 'GSW') | (raw_df['AwayTeam'] == 'GSW')].reset_index()
# df.drop(['index'],1,inplace=True)


In [233]:
# raw_df.head()

In [96]:
len(raw_df)

539265

In [97]:
##train tests split
df = raw_df
sizes = df.groupby(['Date']).size()
mask = np.random.rand(len(sizes)) < 0.8
final_mask = np.array([])
for i in range(len(sizes)):
    final_mask = np.concatenate((final_mask, np.full(sizes[i], mask[i])))
final_mask = final_mask.astype(bool)
df_train = df[final_mask]
df_test = df[~final_mask]

In [100]:
len(df_train), len(df_train.groupby('URL'))

(434659, 922)

In [101]:
df_train.head()

,Date,URL,Location,Time,WinningTeam,Quarter,SecLeft,AwayPlay,HomePlay,AwayTeam,...,AwayMade 3-pt shotTotal,AwayTurnoverTotal,HomeTurnoverTotal,AwayFreeThrowTotal,HomeFreeThrowTotal,AwayMadeFreeThrowTotal,HomeMadeFreeThrowTotal,HomeFinalScore,AwayFinalScore,HomeWin
6575,2019-10-24 00:00:00,/boxscores/201910240DET.html,Little Caesars Arena Detroit Michigan,2021-05-05 19:00:00,ATL,1,720,Jump ball: A. Len vs. A. Drummond (B. Brown ga...,NaN,ATL,...,-3.236354,-3.551278,-3.529433,-3.181216,-3.117167,-2.94465,-2.847606,0.0,1.133333,0
6576,2019-10-24 00:00:00,/boxscores/201910240DET.html,Little Caesars Arena Detroit Michigan,2021-05-05 19:00:00,ATL,1,704,NaN,Turnover by A. Drummond (lost ball; steal by A...,ATL,...,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606,0.0,1.133333,0
6577,2019-10-24 00:00:00,/boxscores/201910240DET.html,Little Caesars Arena Detroit Michigan,2021-05-05 19:00:00,ATL,1,687,D. Hunter misses 2-pt layup from 6 ft,NaN,ATL,...,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606,0.0,1.133333,0
6578,2019-10-24 00:00:00,/boxscores/201910240DET.html,Little Caesars Arena Detroit Michigan,2021-05-05 19:00:00,ATL,1,686,Offensive rebound by A. Len,NaN,ATL,...,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606,0.0,1.133333,0
6579,2019-10-24 00:00:00,/boxscores/201910240DET.html,Little Caesars Arena Detroit Michigan,2021-05-05 19:00:00,ATL,1,686,A. Len makes 2-pt layup from 1 ft,NaN,ATL,...,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606,0.0,1.133333,0


In [102]:
homeLabels = df_train.groupby('URL')['HomeFinalScore'].max()
awayLabels = df_train.groupby('URL')['AwayFinalScore'].max()
df_train.drop(['Location', 'Time', 'WinningTeam', 'Quarter', 'SecLeft',
       'AwayPlay', 'HomePlay', 'AwayScore', 'HomeScore', 'ShotType',
       'ShotOutcome', 'FoulType', 'Fouler', 'Fouled', 'FreeThrowShooter',
       'FreeThrowOutcome', 'FreeThrowNum', 'TurnoverPlayer', 'TurnoverType',
       'TurnoverCause', 'TurnoverCauser', 'hour', 'hour_sin', 'hour_cos',
       'AwayPlayOneHot', 'HomePlayOneHot', '2-pt shot', '3-pt shot', 'HomeWin', 'HomeFinalScore', 'AwayFinalScore'],1,inplace=True)
df_train.head()

/Users/calvingrewal/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Date,URL,AwayTeam,HomeTeam,Home 2-pt shot,Away 2-pt shot,Home 3-pt shot,Away 3-pt shot,HomeMade 2-pt shot,AwayMade 2-pt shot,...,Home 3-pt shotTotal,Away 3-pt shotTotal,HomeMade 3-pt shotTotal,AwayMade 3-pt shotTotal,AwayTurnoverTotal,HomeTurnoverTotal,AwayFreeThrowTotal,HomeFreeThrowTotal,AwayMadeFreeThrowTotal,HomeMadeFreeThrowTotal
6575,2019-10-24 00:00:00,/boxscores/201910240DET.html,ATL,DET,0,0,0,0,0,0,...,-4.836118,-4.85396,-3.199592,-3.236354,-3.551278,-3.529433,-3.181216,-3.117167,-2.94465,-2.847606
6576,2019-10-24 00:00:00,/boxscores/201910240DET.html,ATL,DET,0,0,0,0,0,0,...,-4.836118,-4.85396,-3.199592,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606
6577,2019-10-24 00:00:00,/boxscores/201910240DET.html,ATL,DET,0,1,0,0,0,0,...,-4.836118,-4.85396,-3.199592,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606
6578,2019-10-24 00:00:00,/boxscores/201910240DET.html,ATL,DET,0,0,0,0,0,0,...,-4.836118,-4.85396,-3.199592,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606
6579,2019-10-24 00:00:00,/boxscores/201910240DET.html,ATL,DET,0,1,0,0,0,1,...,-4.836118,-4.85396,-3.199592,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606


In [103]:
df_train.columns, len(df_train), len(df_train.columns)

(Index(['Date', 'URL', 'AwayTeam', 'HomeTeam', 'Home 2-pt shot',
        'Away 2-pt shot', 'Home 3-pt shot', 'Away 3-pt shot',
        'HomeMade 2-pt shot', 'AwayMade 2-pt shot', 'HomeMade 3-pt shot',
        'AwayMade 3-pt shot', 'AwayTurnover', 'HomeTurnover', 'AwayFreeThrow',
        'HomeFreeThrow', 'AwayMadeFreeThrow', 'HomeMadeFreeThrow',
        'GameTimeLeft', 'Home 2-pt shotTotal', 'Away 2-pt shotTotal',
        'HomeMade 2-pt shotTotal', 'AwayMade 2-pt shotTotal',
        'Home 3-pt shotTotal', 'Away 3-pt shotTotal', 'HomeMade 3-pt shotTotal',
        'AwayMade 3-pt shotTotal', 'AwayTurnoverTotal', 'HomeTurnoverTotal',
        'AwayFreeThrowTotal', 'HomeFreeThrowTotal', 'AwayMadeFreeThrowTotal',
        'HomeMadeFreeThrowTotal'],
       dtype='object'), 434659, 33)

In [104]:
df_train.groupby('URL').size().max()

605

In [105]:
len(df_train.groupby('URL'))

922

In [113]:
game_urls = df_train['URL'].unique()
sampler = BatchSampler(RandomSampler(game_urls), 4, drop_last=False)
game_urls[:5]

array(['/boxscores/201910240DET.html', '/boxscores/201910240GSW.html',
       '/boxscores/201910240HOU.html', '/boxscores/201910250BOS.html',
       '/boxscores/201910250BRK.html'], dtype=object)

In [231]:
# for indices in sampler:
#     print(indices)

In [108]:
cols = ['Home 2-pt shot', 'Away 2-pt shot',
        'Home 3-pt shot', 'Away 3-pt shot', 'HomeMade 2-pt shot',
        'AwayMade 2-pt shot', 'HomeMade 3-pt shot', 'AwayMade 3-pt shot',
        'AwayTurnover', 'HomeTurnover', 'AwayFreeThrow', 'HomeFreeThrow',
        'AwayMadeFreeThrow', 'HomeMadeFreeThrow', 'GameTimeLeft',
        'Home 2-pt shotTotal', 'Away 2-pt shotTotal', 'HomeMade 2-pt shotTotal',
        'AwayMade 2-pt shotTotal', 'Home 3-pt shotTotal', 'Away 3-pt shotTotal',
        'HomeMade 3-pt shotTotal', 'AwayMade 3-pt shotTotal',
        'AwayTurnoverTotal', 'HomeTurnoverTotal', 'AwayFreeThrowTotal',
        'HomeFreeThrowTotal', 'AwayMadeFreeThrowTotal',
        'HomeMadeFreeThrowTotal']

In [116]:
df_train['listified'] = df_train[cols].apply(list, axis=1).aggregate(list)
df_train.head()

/Users/calvingrewal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Date,URL,AwayTeam,HomeTeam,Home 2-pt shot,Away 2-pt shot,Home 3-pt shot,Away 3-pt shot,HomeMade 2-pt shot,AwayMade 2-pt shot,...,Away 3-pt shotTotal,HomeMade 3-pt shotTotal,AwayMade 3-pt shotTotal,AwayTurnoverTotal,HomeTurnoverTotal,AwayFreeThrowTotal,HomeFreeThrowTotal,AwayMadeFreeThrowTotal,HomeMadeFreeThrowTotal,listified
6575,2019-10-24 00:00:00,/boxscores/201910240DET.html,ATL,DET,0,0,0,0,0,0,...,-4.85396,-3.199592,-3.236354,-3.551278,-3.529433,-3.181216,-3.117167,-2.94465,-2.847606,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6576,2019-10-24 00:00:00,/boxscores/201910240DET.html,ATL,DET,0,0,0,0,0,0,...,-4.85396,-3.199592,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6577,2019-10-24 00:00:00,/boxscores/201910240DET.html,ATL,DET,0,1,0,0,0,0,...,-4.85396,-3.199592,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6578,2019-10-24 00:00:00,/boxscores/201910240DET.html,ATL,DET,0,0,0,0,0,0,...,-4.85396,-3.199592,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6579,2019-10-24 00:00:00,/boxscores/201910240DET.html,ATL,DET,0,1,0,0,0,1,...,-4.85396,-3.199592,-3.236354,-3.551278,-3.285884,-3.181216,-3.117167,-2.94465,-2.847606,"[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


In [194]:
train_data = df_train.groupby('URL')['listified'].aggregate(list)
train_data = train_data.tolist()

In [195]:
len(train_data),len(train_data[0][0]), train_data[0], 

(922,
 29,
 [[0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   -6.540084370390012,
   -6.596201027211362,
   -5.320539090307989,
   -5.335444870441754,
   -4.8361179420976566,
   -4.853960209634527,
   -3.1995916618626032,
   -3.2363539286603986,
   -3.5512782926657405,
   -3.529433115908964,
   -3.18121621277943,
   -3.117166561055918,
   -2.9446496121628107,
   -2.847605962886945],
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.9944444444444444,
   -6.540084370390012,
   -6.596201027211362,
   -5.320539090307989,
   -5.335444870441754,
   -4.8361179420976566,
   -4.853960209634527,
   -3.1995916618626032,
   -3.2363539286603986,
   -3.5512782926657405,
   -3.285884332312976,
   -3.18121621277943,
   -3.117166561055918,
   -2.9446496121628107,
   -2.847605962886945],
  [0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,


In [196]:
len(homeLabels)

922

In [197]:
a = [[-1000] * 5]
a * 2

[[-1000, -1000, -1000, -1000, -1000], [-1000, -1000, -1000, -1000, -1000]]

In [198]:
pad_length = 619
vec_length = len(train_data[0][0])
blank_pad = [[-1000] * vec_length]
masks = [] # index of the last unpadded input for each game
for game in train_data:
#     print(len(game), len(game[0]))
    padding = pad_length - len(game)
    masks.append(len(game)-1)
    game.extend(blank_pad * padding )

In [199]:
train_data = torch.tensor(train_data)
train_data.shape

torch.Size([922, 619, 29])

In [202]:
train_data[0,masks[0]+1, :]

tensor([-1000., -1000., -1000., -1000., -1000., -1000., -1000., -1000., -1000.,
        -1000., -1000., -1000., -1000., -1000., -1000., -1000., -1000., -1000.,
        -1000., -1000., -1000., -1000., -1000., -1000., -1000., -1000., -1000.,
        -1000., -1000.])

In [203]:
masks = np.array(masks)
def build_batch(indices):
    batch_data = train_data[indices]
    batch_labels_home = torch.tensor(homeLabels[indices].tolist())
    batch_labels_away = torch.tensor(awayLabels[indices].tolist())
#     print('hi')
    batch_masks = masks[indices]
    return batch_data, batch_labels_home, batch_labels_away, batch_masks

In [229]:
# in_size = 32
batch_size = 4
time_steps = 619

num_epochs = 20
model = LSTM_Model(train_data.shape[-1], 512)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [218]:
with torch.no_grad():
    for indices in BatchSampler(RandomSampler(game_urls), batch_size, drop_last=False):
        batch_data, batch_labels_home, batch_labels_away, batch_masks = build_batch(indices)
#             print(type(batch_data), type(batch_labels_home), type(batch_labels_away))
        output_seq = model(batch_data)
        print(output_seq.shape)

#         last_output = output_seq[:, -1,:]
            B = last_output.shape[0]
            last_output = torch.zeros(B, 2) # (B, 2)
    #         test = torch.zeros(B, 2)
            for i in range(B):
                last_output[i] = output_seq[i, batch_masks[i], :]            
        print(last_output.shape, output_seq)
#       print(batch_labels_home.shape, batch_labels_away.shape,)
        target = torch.hstack(
                    (batch_labels_home.reshape(-1, 1), batch_labels_away.reshape(-1, 1))
                )
        loss = criterion(last_output, target)
        print(loss.item())
#             print(torch.hstack((batch_labels_home.reshape(-1, 1), batch_labels_away.reshape(-1, 1))))

torch.Size([8, 619, 2])
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.9114,
         0.1077,  0.6055,  0.5395, -0.8664,  0.3621, -0.6072, -0.5854,  0.1382,
        -1.0939,  0.9918,  1.2651,  0.3845,  1.2244])
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.4307,
         0.4734,  1.3462,  0.7290, -1.0082, -1.6115, -1.3849, -1.6458,  0.1382,
         0.3673,  0.0181, -0.0629,  0.3845, -0.1851])
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.7912,
         1.6923,  0.9758,  1.6766, -0.0158, -1.3296,  0.4298, -0.8505,  1.6139,
        -1.0939,  0.2963, -0.0629,  0.3845, -0.1851])
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.000

KeyboardInterrupt: 

In [230]:
model.train()

for epoch in range(num_epochs):
    for i, indices in enumerate(BatchSampler(RandomSampler(game_urls[:10]), batch_size, drop_last=False)):
        batch_data, batch_labels_home, batch_labels_away, batch_masks = build_batch(indices)
        # batch_data has shape(B, 619, 19), the 619 is the padded sequence length, and the 19 is the feature vector for each timesetep
        output_seq = model(batch_data)
        
#         last_output = output_seq[:,-1,:]
        B = output_seq.shape[0]
        last_output = torch.zeros(B, 2) # sinnce we have a many-to-one problem, we only want the last element in each sequence
        for j in range(B): # spent way too long tryna figure out the cool way to do this, but i guess a for loop works
            last_output[j] = output_seq[j, batch_masks[j], :]   
#         print(last_output.shape)
        
        target = torch.hstack(
            (batch_labels_home.reshape(-1, 1), batch_labels_away.reshape(-1, 1))
        )
        loss = criterion(last_output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      
        print(f'epoch: {epoch}, i: {i}', loss.item())
        print('actual: ', target * 15 + 100)
        print('predicted: ', last_output * 15 + 100)
# for i in range(len(df_train)):
#     # Forward pass: Compute predicted y by passing x to the model
#     x = torch.FloatTensor(df_train.iloc[i]).unsqueeze(0).unsqueeze(0)
#     y = torch.FloatTensor((homeLabels.iloc[i], awayLabels.iloc[i])).unsqueeze(0).unsqueeze(0)
#     y_pred = model(x)

# #     # Compute and print loss
#     loss = criterion(y_pred, y)
#     if i % 100 == 99:
#         print(i, loss.item())

# #     # Zero gradients, perform a backward pass, and update the weights.
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

epoch: 0, i: 0 0.7911989688873291
actual:  tensor([[113., 109.],
        [ 95.,  86.],
        [108., 107.],
        [116., 123.]])
predicted:  tensor([[ 99.1105, 100.1921],
        [ 99.0608,  99.9513],
        [ 99.1786,  99.6687],
        [ 98.7831, 100.3164]], grad_fn=<AddBackward0>)
epoch: 0, i: 1 1.5728659629821777
actual:  tensor([[112., 106.],
        [111., 117.],
        [102., 110.],
        [122., 141.]])
predicted:  tensor([[ 99.5217, 100.0126],
        [ 99.5522, 100.8458],
        [ 99.0930, 100.3664],
        [ 99.8806, 100.2929]], grad_fn=<AddBackward0>)
epoch: 0, i: 2 0.7587997913360596
actual:  tensor([[ 99., 121.],
        [100., 117.]])
predicted:  tensor([[ 99.7392, 100.8281],
        [ 99.3492, 100.4156]], grad_fn=<AddBackward0>)
epoch: 1, i: 0 1.4038746356964111
actual:  tensor([[100., 117.],
        [122., 141.],
        [102., 110.],
        [ 95.,  86.]])
predicted:  tensor([[ 99.3315, 100.9056],
        [101.2139, 101.8959],
        [ 99.3333, 100.7437],
   

epoch: 10, i: 2 0.3365461826324463
actual:  tensor([[122., 141.],
        [108., 107.]])
predicted:  tensor([[116.1056, 125.3773],
        [103.1112, 107.4240]], grad_fn=<AddBackward0>)
epoch: 11, i: 0 0.07959382236003876
actual:  tensor([[116., 123.],
        [ 95.,  86.],
        [102., 110.],
        [112., 106.]])
predicted:  tensor([[108.9475, 123.0001],
        [ 98.5304,  92.9380],
        [100.7842, 107.5970],
        [107.8694, 108.9355]], grad_fn=<AddBackward0>)
epoch: 11, i: 1 0.10847679525613785
actual:  tensor([[122., 141.],
        [ 99., 121.],
        [100., 117.],
        [108., 107.]])
predicted:  tensor([[117.6566, 133.7555],
        [104.5939, 117.2013],
        [102.2052, 110.0086],
        [104.0184, 109.9313]], grad_fn=<AddBackward0>)
epoch: 11, i: 2 0.11625410616397858
actual:  tensor([[111., 117.],
        [113., 109.]])
predicted:  tensor([[114.0748, 124.0055],
        [106.8062, 106.2189]], grad_fn=<AddBackward0>)
epoch: 12, i: 0 0.12098345160484314
actual:  

In [91]:
df_train['HomeFinalScore']

KeyError: 'HomeFinalScore'

In [167]:
testHomeLabels = df_test['HomeFinalScore']
testAwayLabels = df_test['AwayFinalScore']
testHomeWins = (df_test['AwayFinalScore'] > df_test['HomeFinalScore']).astype(int)

df_test.drop(['AwayTeam', 'HomeTeam','Date','HomeFinalScore','AwayFinalScore'],1,inplace=True)

In [197]:
start_test = df_test.loc[df['GameTimeLeft'] == 1]
testHomeLabels = start_test['HomeFinalScore']
testAwayLabels = start_test['AwayFinalScore']
testHomeWins = (start_test['AwayFinalScore'] > start_test['HomeFinalScore']).astype(int)
start_test.drop(['AwayTeam', 'HomeTeam','Date','HomeFinalScore','AwayFinalScore'],1,inplace=True)

C:\Users\eric_\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [198]:
len(testHomeWins)

15

In [208]:
model.eval()
preds = []
for i in range(len(start_test)):
    x = torch.FloatTensor(start_test.iloc[i]).unsqueeze(0).unsqueeze(0)
    y = torch.FloatTensor((testHomeLabels.iloc[i], testAwayLabels.iloc[i]))
    
    pred = model(x)
    
    if (pred[0,0,0].item() > pred[0,0,1].item()):
        preds.append(True)
    else:
        preds.append(False)
preds

    

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [209]:
testHomeWins

0       1
481     0
947     0
1469    0
1958    0
2422    1
2874    1
3347    0
3833    1
4419    0
4837    1
5298    1
5749    1
6243    1
6708    0
dtype: int32

In [210]:
preds

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]